In [1]:
import tensorflow as tf
import edward as ed
from sklearn.datasets import make_moons
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import math

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
class Gumbel:
    def __init__(self, shape, logits, name='Gumbel'):
        self.shape = shape
        self.name = name
        with tf.name_scope(self.name):
            with tf.name_scope('normalized_logits'):
                self.logits = logits - tf.log(tf.reduce_sum(tf.exp(logits), axis=-1))[...,tf.newaxis]
    def sample(self, us=None, argmax=None):
        with tf.name_scope(self.name):
            if us is None:
                us = tf.random_uniform(self.shape, minval=1e-3, maxval=1-1e-3)
            self.uniform_sample = us
            
            if argmax is None:
                gb = -tf.log(-tf.log(us)) + self.logits
            else:
                upper = -tf.log(-tf.log(us))
                argmax_ix = tf.argmax(argmax, axis=-1)
                upper_samples = tf.reduce_sum(us*argmax, axis=-1)
                gb = -tf.log(-tf.log(us)/tf.exp(self.logits) - tf.log(upper_samples)[...,tf.newaxis])
                gb = upper*argmax + gb*(1-argmax)
#                 return upper*argmax
            return gb

In [3]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
# X, y = make_moons(30000, noise=0.2)
# X = X.astype('float32')
# y = y.astype('float32')
# dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = mnist.train
dataset = tf.data.Dataset.from_tensor_slices((dataset.images, dataset.labels))

In [5]:
dataset = dataset.shuffle(1000).repeat().batch(100)

In [6]:
diter = dataset.make_one_shot_iterator()

In [7]:
def dense(x, dim, name, activation=tf.nn.tanh):
    with tf.variable_scope(name, initializer=tf.random_normal_initializer(stddev=0.01)):
        W = tf.get_variable('W', [x.shape[1], dim])
        b = tf.get_variable('b', [1,dim])
        logit = tf.matmul(x, W) + b
        return activation(logit)
    
class VAE:
    def __init__(self, dim, hdim, temp, name='VAE', config=[128,64,32]):
        self.dim = dim
        self.hdim = hdim
        self.name = name
        self.config = config
        self.temp = temp
        with tf.variable_scope(self.name) as scope:
            self.scope = scope
    
    def priorkl(self, encoded, encoded_logits):
        with tf.name_scope('priorkl'):
            labels = tf.ones_like(encoded)/tf.cast(2, tf.float32)
            xent = tf.reduce_sum(encoded*tf.log(labels), axis=[-1,-2], name='xent')
#             nent = -tf.nn.softmax_cross_entropy_with_logits_v2(labels=encoded, logits=encoded_logits)
#             nent = tf.reduce_sum(nent, axis=-1)
            nent = tf.reduce_sum(encoded*tf.log(encoded), axis=[-1,-2], name='nent')
            with tf.control_dependencies([tf.check_numerics(x, message='priorkl_numerics_{}'.format(x.name)) for x in [xent, nent]]):
                loss = -xent + nent
        return loss
            
    def encode(self, x, hard=False, uniform_sample=None, condition=None):
        with tf.variable_scope('encoder', reuse=tf.AUTO_REUSE) as scope:
            self.encoder_scope = scope
            for i, out_dim in enumerate(self.config):
                x = dense(x, out_dim, name='d{}'.format(i))
                
            with tf.variable_scope('latent_inf'):
                logits = dense(x, self.hdim*2, name='logits', activation=lambda x: x)
                logits = tf.reshape(logits, [tf.shape(x)[0], self.hdim, 2])
                self.logits = logits
                
                self.center_loss = 1e-2*tf.reduce_mean(tf.square(tf.log(tf.reduce_sum(tf.exp(logits), axis=-1))))
                
                gd = Gumbel(tf.shape(logits), logits=logits)
                encoded_gumb = gd.sample(us=uniform_sample, argmax=condition)
                self.uniform_sample = gd.uniform_sample
                
                encoded = tf.nn.softmax(encoded_gumb/self.temp)
                self.kl_loss = self.priorkl(tf.nn.softmax(self.logits), self.logits)
                encoded_soft = encoded

                encoded_hard = tf.distributions.Multinomial(1., logits=self.logits).sample()
                
                return encoded_soft, encoded_hard
    
    def decode(self, x):
        with tf.variable_scope('decoder', reuse=tf.AUTO_REUSE) as scope:
            self.decoder_scope = scope
            x = tf.reshape(x, [tf.shape(x)[0], self.hdim*2])
            for i, out_dim in enumerate(self.config[::-1]):
                x = dense(x, out_dim, name='d{}'.format(i))
            x = dense(x, self.dim, name='restoration', activation=lambda x: x)
            return x

In [8]:
data, y = diter.get_next()#tf.Variable(X, trainable=False)

In [9]:
vae_hdim = 10

with tf.variable_scope('ConVar') as con_var_scope:
    pretemperature = tf.get_variable('pretemperature', shape=[1,vae_hdim,2],
                                     initializer=tf.constant_initializer(math.log(.3)), trainable=False)
    preeta = tf.get_variable('peeta', shape=(),
                             initializer=tf.constant_initializer(0.), trainable=False)
    with tf.variable_scope('nvil_baseline') as nvil_scope:
        d1 = dense(data, 128, 'd1')
        nvil_baseline = dense(data, 1, 'out')[:,0]
    
    temperature = tf.exp(pretemperature)
    tf.summary.histogram('temperature', temperature)
    eta = tf.exp(preeta)

    #784
vae = VAE(784, vae_hdim, temp=temperature, config=[128,128])

In [10]:
data

<tf.Tensor 'IteratorGetNext:0' shape=(?, 784) dtype=float32>

In [11]:
_, encoded_hard = vae.encode(data)
encoded_soft, _ = vae.encode(data)
encoded_soft_conditioned, _ = vae.encode(data, condition=encoded_hard)

In [12]:
def build_fc(inp):
    d1 = dense(inp, 128, 'd1')
    out = dense(d1, 1, 'out', activation=lambda x: x)
    return out[:,0]

In [13]:
with tf.variable_scope(con_var_scope):
    with tf.variable_scope('relax') as relax_scope:
        flat_uncond = tf.reshape(encoded_soft, [-1, vae_hdim*2])
        flat_cond = tf.reshape(encoded_soft_conditioned, [-1, vae_hdim*2])
        
        relax_uncond = build_fc(flat_uncond)
        relax_scope.reuse_variables()
        relax_cond = build_fc(flat_cond)

In [14]:
vae.logits

<tf.Tensor 'encoder_2/latent_inf/Reshape:0' shape=(?, 10, 2) dtype=float32>

In [15]:
sample_logp = tf.distributions.Multinomial(1., logits=vae.logits).log_prob(encoded_hard)
sample_logp = tf.reduce_sum(sample_logp, axis=-1)

In [16]:
decoded_soft = vae.decode(encoded_soft)
decoded_hard = vae.decode(encoded_hard)
decoded_soft_conditioned = vae.decode(encoded_soft_conditioned)

In [17]:
def rec_loss(x, dec):
    sigma = tf.constant(0.6)
    loss = tf.reduce_sum(tf.square(x - dec), axis=-1)/(2*tf.square(sigma)) - tf.log(2*3.14*sigma)
    return loss
def get_elbo(decoded):
    loss = rec_loss(data, decoded) + vae.kl_loss
    return -loss

In [18]:
soft_elbo = get_elbo(decoded_soft)
hard_elbo = get_elbo(decoded_hard)
soft_elbo_conditioned = get_elbo(decoded_soft_conditioned)

In [19]:
soft_elbo

<tf.Tensor 'Neg:0' shape=(?,) dtype=float32>

In [20]:
tf.summary.scalar('soft_elbo', tf.reduce_mean(soft_elbo))
tf.summary.scalar('hard_elbo', tf.reduce_mean(hard_elbo))

<tf.Tensor 'hard_elbo:0' shape=() dtype=string>

In [21]:
tf.summary.scalar('eta', eta)
tf.summary.scalar('mean_temperature', tf.reduce_mean(temperature))

<tf.Tensor 'mean_temperature:0' shape=() dtype=string>

In [22]:
graph = tf.get_default_graph()
encoder_vars = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=vae.encoder_scope.name)

In [23]:
# stoped_grad_input = hard_elbo - eta*soft_elbo_conditioned
soft_control_cond = eta*soft_elbo_conditioned + relax_cond
soft_control_uncond = eta*soft_elbo + relax_uncond - eta*soft_elbo_conditioned - relax_cond

stoped_grad = hard_elbo - soft_control_cond - nvil_baseline
# stoped_grad = tf.stop_gradient(hard_elbo - eta*soft_elbo_conditioned)

target = -sample_logp*stoped_grad - soft_control_uncond + vae.center_loss
# mean_elbo = tf.Variable(-40.)
# tf.summary.scalar('mean_elbo', mean_elbo)
# target = -sample_logp*tf.stop_gradient(hard_elbo - mean_elbo)
# with tf.control_dependencies([target]):
#     mean_elbo_op = mean_elbo.assign(mean_elbo*0.98 + tf.reduce_mean(hard_elbo)*0.02)
target = tf.reduce_mean(target)

In [24]:
nvil_baseline

<tf.Tensor 'ConVar/nvil_baseline/strided_slice:0' shape=(?,) dtype=float32>

In [25]:
sample_logp, hard_elbo, soft_elbo

(<tf.Tensor 'Sum:0' shape=(?,) dtype=float32>,
 <tf.Tensor 'Neg_1:0' shape=(?,) dtype=float32>,
 <tf.Tensor 'Neg:0' shape=(?,) dtype=float32>)

In [26]:
# logp_grad = tf.gradients(sample_logp, encoder_vars)
# soft_elbo_grad = tf.gradients(soft_elbo, encoder_vars)

# grads_encoder = [-lpg*(hard_elbo-eta*soft_elbo)-eta*seg for lpg, seg in zip(logp_grad, soft_elbo_grad)]

grads_encoder = tf.gradients(target, encoder_vars, stop_gradients=stoped_grad)
# grads_encoder = tf.gradients(-soft_elbo, encoder_vars)

In [27]:
with tf.name_scope('controls'):
    tmp = [tf.reduce_sum(tf.square(x)) for x in grads_encoder]
    with tf.name_scope('denum'):
        denum = tf.reduce_sum([tf.reduce_sum(tf.shape(x)) for x in grads_encoder])
        denum = tf.cast(denum, tf.float32)
    controls_loss = tf.reduce_sum(tmp)/denum
    tf.summary.scalar('controls_loss', controls_loss)

In [28]:
decoder_vars = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=vae.decoder_scope.name)

In [29]:
grads_decoder = tf.gradients(-hard_elbo, decoder_vars)

In [30]:
# pretemperature, preeta, controls_loss

In [31]:
with tf.name_scope('var_reductions'):
    nvil_vars = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=nvil_scope.name)
    relax_vars = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=relax_scope.name)

    loss = controls_loss
    reds = [preeta, pretemperature] + nvil_vars + relax_vars
    grads_reds = tf.gradients(loss, reds)
gvs_reds = [(g,v) for g,v in zip(grads_reds, reds)]

In [32]:
variables = encoder_vars + decoder_vars
grads = grads_encoder + grads_decoder
gvs = [(g,v) for g,v in zip(grads, variables)]

In [33]:
with tf.control_dependencies([tf.check_numerics(g, message='Numerics trouble') for g,x in gvs]):
    main_op = tf.train.AdamOptimizer(0.001).apply_gradients(gvs)
red_op = tf.train.AdamOptimizer(0.002).apply_gradients(gvs_reds)

In [34]:
train_op = [main_op, red_op]

In [35]:
# !rm -R /tmp/tfdbg

In [36]:
sess = tf.InteractiveSession()
writer = tf.summary.FileWriter('/tmp/tfdbg/relax_mnist100_red0.002_1e-2_center_moarparams', tf.get_default_graph())
init = tf.global_variables_initializer()

In [37]:
init.run()

In [38]:
gvs_reds[1][0].eval()

array([[[-5.9056909e-10,  1.4692283e-09],
        [ 1.7563994e-09, -6.7957567e-10],
        [ 2.1706499e-09, -1.0659303e-09],
        [ 1.2371822e-09, -2.1487923e-09],
        [ 4.2185575e-09, -3.8833479e-09],
        [ 4.1620627e-09, -4.6238919e-09],
        [ 5.6817484e-10, -1.0693981e-09],
        [-1.5251654e-09,  1.7674333e-09],
        [-3.8427608e-09,  6.2509593e-09],
        [-2.2670090e-09,  1.0581721e-09]]], dtype=float32)

In [39]:
all_summaries = tf.summary.merge_all()

In [40]:
step = 0

In [41]:
for step in range(step, 600000):
    _, sm = sess.run([train_op, all_summaries])
    writer.add_summary(sm, global_step=step)

KeyboardInterrupt: 

In [42]:
prior = tf.distributions.Multinomial(1., logits=tf.zeros([10, vae.hdim, 2])).sample()
generated = vae.decode(prior)

In [ ]:
tt = np.reshape(generated.eval(), [-1,28,28])

In [ ]:
tt.shape

In [ ]:
for i in range(10):
    plt.imshow(tt[i])
    plt.show()

In [ ]:
pretemperature.assign(math.log(.4)).eval()